In [5]:
import pandas as pd

state_data = pd.read_csv('datasets/merged_state_data.csv')
weather_data = pd.read_csv('datasets/weather_monthly_state_aggregates.csv')
wildfire_data = pd.read_csv('datasets/wildfire_sizes_before_2010.csv')

print(len(state_data))
print(len(weather_data))
print(len(wildfire_data))

50
11775
9632


In [17]:
state_data = state_data.rename(columns={'State': 'STATE', 'mean_elevation': 'MEAN_ELEVATION', 'Land Area (sq mi)': 'LAND_AREA', 'Water Area (sq mi)': 'WATER_AREA', 'Total Area (sq mi)': 'TOTAL_AREA', 'Percentage of Federal Land': 'PERCENTAGE_FEDERAL_LAND', 'Urbanization Rate (%)': 'URBANIZATION_RATE'})
state_data.head()

,STATE,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE
0,AK,580,570641,94743,665384,60.9%,64.9
1,AL,150,50645,1775,52420,2.7%,57.7
2,AR,200,52035,1143,53179,9.4%,55.5
3,AZ,1250,113594,396,113990,38.6%,89.3
4,CA,880,155779,7916,163695,45.4%,94.2


In [89]:
weather_data = weather_data.rename(columns={'State': 'STATE', 'year_month': 'PERIOD'})
weather_data.head()

,STATE,PERIOD,PRCP,EVAP,TMIN,TMAX
0,AK,1992-05,5.36,44.17,-56.0,233.0
1,AK,1992-06,14.03,39.27,-17.0,322.0
2,AK,1992-07,16.78,30.91,28.0,300.0
3,AK,1992-08,10.46,20.14,-33.0,267.0
4,AK,1992-09,17.25,15.27,-61.0,183.0


In [90]:
weather_data.describe()

,PRCP,EVAP,TMIN,TMAX
count,11775.000000,11775.000000,11775.000000,11775.000000
mean,22.792403,62.285918,11.948875,318.659788
std,18.153277,185.920998,82.541805,71.463314
min,0.000000,-96.870000,-456.000000,-167.000000
25%,9.570000,31.280000,-39.000000,283.000000
50%,19.820000,48.400000,6.000000,328.000000
75%,32.000000,64.895000,67.000000,361.000000
max,259.000000,15445.700000,233.000000,539.000000


In [91]:
wildfire_data = wildfire_data.rename(columns={'month': 'PERIOD', 'total_fire_size': 'TOTAL_FIRE_SIZE'})
wildfire_data.head()

,STATE,PERIOD,TOTAL_FIRE_SIZE
0,AK,1992-04,10.1
1,AK,1992-05,4309.5
2,AK,1992-06,86460.6
3,AK,1992-07,48578.5
4,AK,1992-08,3321.8


In [93]:
wildfire_data.describe()

,TOTAL_FIRE_SIZE
count,9.632000e+03
mean,1.064928e+04
std,8.248342e+04
min,1.000000e-02
25%,2.400000e+01
50%,2.997650e+02
75%,2.439590e+03
max,4.779145e+06


In [95]:
weather_wildfire_data = pd.merge(wildfire_data, weather_data, on=['STATE', 'PERIOD'], how='left')
print(len(weather_wildfire_data))
# weather_wildfire_data.head()
weather_wildfire_data.describe()

9632


,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX
count,9.632000e+03,6583.000000,6583.000000,6583.000000,6583.000000
mean,1.064928e+04,21.221559,50.931341,10.677199,328.046939
std,8.248342e+04,15.636771,23.654458,73.950807,61.079450
min,1.000000e-02,0.000000,-6.860000,-456.000000,-83.000000
25%,2.400000e+01,9.775000,34.370000,-39.000000,289.000000
50%,2.997650e+02,18.790000,49.320000,-6.000000,328.000000
75%,2.439590e+03,29.480000,63.150000,56.000000,367.000000
max,4.779145e+06,213.000000,392.330000,217.000000,539.000000


In [96]:
state_weather_wildfire_data = pd.merge(weather_wildfire_data, state_data, on=['STATE'], how='left')
print(len(state_weather_wildfire_data))
state_weather_wildfire_data.head()

9632


,STATE,PERIOD,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE
0,AK,1992-04,10.1,NaN,NaN,NaN,NaN,580,570641,94743,665384,60.9%,64.9
1,AK,1992-05,4309.5,5.36,44.17,-56.0,233.0,580,570641,94743,665384,60.9%,64.9
2,AK,1992-06,86460.6,14.03,39.27,-17.0,322.0,580,570641,94743,665384,60.9%,64.9
3,AK,1992-07,48578.5,16.78,30.91,28.0,300.0,580,570641,94743,665384,60.9%,64.9
4,AK,1992-08,3321.8,10.46,20.14,-33.0,267.0,580,570641,94743,665384,60.9%,64.9


In [29]:
state_weather_wildfire_data.to_csv('datasets/combined_dataset.csv', index=False)

#### More data cleaning

In [106]:
# load dataset
dataset_loaded = pd.read_csv('datasets/combined_dataset.csv')
dataset_loaded.head()

# convert date to sin encoded month / convert strings to numeric
dataset_loaded['PERIOD'] = pd.to_datetime(dataset_loaded['PERIOD'], format='%Y-%m')
dataset_loaded['MONTH'] = dataset_loaded['PERIOD'].dt.month
dataset_loaded['YEAR'] = dataset_loaded['PERIOD'].dt.year
dataset_loaded['MONTH_SIN'] = np.sin(2 * np.pi * dataset_loaded['MONTH'] / 12)
dataset_loaded['PERCENTAGE_FEDERAL_LAND'] = pd.to_numeric(
    dataset_loaded['PERCENTAGE_FEDERAL_LAND'].astype(str).str.rstrip('%').astype(float),
    errors='coerce'
)
# dataset_loaded = dataset_loaded.drop(columns=['PERIOD', 'MONTH', 'YEAR'])

dataset_loaded.head()
dataset_loaded.describe()


target_feature = 'MONTH'
features_to_check = ['PRCP', 'EVAP', 'TMIN', 'TMAX']

null_counts_by_target = dataset_loaded.groupby(target_feature)[features_to_check].apply(lambda group: group.isnull().sum())
print("\nNull counts in defined features for each target value:")
print(null_counts_by_target)



Null counts in defined features for each target value:
       PRCP  EVAP  TMIN  TMAX
MONTH                        
1       438   438   438   438
2       494   494   494   494
3       548   548   548   548
4       232   232   232   232
5        85    85    85    85
6        70    70    70    70
7        77    77    77    77
8        71    71    71    71
9        72    72    72    72
10      127   127   127   127
11      406   406   406   406
12      429   429   429   429


In [107]:
for feature in features_to_check:
    # Count the missing values per target group, calculate mean per group
    missing_counts = dataset_loaded.groupby(target_feature)[feature].apply(lambda x: x.isna().sum())
    group_means = dataset_loaded.groupby(target_feature)[feature].mean()

    # Consider only groups that actually have missing values
    groups_with_missing = missing_counts[missing_counts > 0]

    if groups_with_missing.sum() > 0:
        weighted_avg = (group_means[groups_with_missing.index] * groups_with_missing).sum() / groups_with_missing.sum()
        print(f"Weighted average for {feature}: {weighted_avg}")

        # Fill missing values with the computed weighted average
        dataset_loaded[feature].fillna(weighted_avg, inplace=True)
    else:
        print(f"No missing values for {feature}.")


null_counts_by_target = dataset_loaded.groupby(target_feature)[features_to_check].apply(lambda group: group.isnull().sum())
print("\nNull counts in defined features for each target value:")
print(null_counts_by_target)


Weighted average for PRCP: 22.187598769705254
Weighted average for EVAP: 36.501526426424135
Weighted average for TMIN: -29.51584264995739
Weighted average for TMAX: 282.5082348568138

Null counts in defined features for each target value:
       PRCP  EVAP  TMIN  TMAX
MONTH                        
1         0     0     0     0
2         0     0     0     0
3         0     0     0     0
4         0     0     0     0
5         0     0     0     0
6         0     0     0     0
7         0     0     0     0
8         0     0     0     0
9         0     0     0     0
10        0     0     0     0
11        0     0     0     0
12        0     0     0     0


/tmp/ipykernel_173000/3224680622.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset_loaded[feature].fillna(weighted_avg, inplace=True)


In [109]:
dataset_loaded = dataset_loaded.drop(columns=['MONTH', 'YEAR', 'PERIOD'])
dataset_loaded.describe()

,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE,MONTH_SIN
count,9.632000e+03,9632.000000,9632.000000,9632.000000,9632.000000,9632.000000,9632.000000,9632.000000,9632.000000,9632.000000,9632.000000,9.632000e+03
mean,1.064928e+04,21.527358,46.363598,-2.045868,313.631708,546.721034,74150.715947,5075.553156,79226.212313,16.331676,72.499201,-2.842887e-03
std,8.248342e+04,12.934587,20.674800,63.929354,54.756974,538.988922,78942.638002,12927.298639,88882.864369,20.535084,13.565915,7.162742e-01
min,1.000000e-02,0.000000,-6.860000,-456.000000,-83.000000,20.000000,1034.000000,192.000000,1545.000000,0.300000,35.100000,-1.000000e+00
25%,2.400000e+01,14.090000,36.501526,-29.515843,282.508235,180.000000,40861.000000,791.000000,42775.000000,2.400000,63.200000,-8.660254e-01
50%,2.997650e+02,22.187599,36.501526,-29.515843,294.000000,300.000000,55519.000000,1509.000000,65496.000000,5.400000,71.900000,1.224647e-16
75%,2.439590e+03,24.212500,56.270000,28.000000,350.000000,670.000000,82170.000000,4537.000000,86936.000000,28.600000,85.600000,8.660254e-01
max,4.779145e+06,213.000000,392.330000,217.000000,539.000000,2070.000000,570641.000000,94743.000000,665384.000000,80.100000,94.200000,1.000000e+00


In [110]:
dataset_loaded.head()

,STATE,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE,MONTH_SIN
0,AK,10.1,22.187599,36.501526,-29.515843,282.508235,580,570641,94743,665384,60.9,64.9,8.660254e-01
1,AK,4309.5,5.360000,44.170000,-56.000000,233.000000,580,570641,94743,665384,60.9,64.9,5.000000e-01
2,AK,86460.6,14.030000,39.270000,-17.000000,322.000000,580,570641,94743,665384,60.9,64.9,1.224647e-16
3,AK,48578.5,16.780000,30.910000,28.000000,300.000000,580,570641,94743,665384,60.9,64.9,-5.000000e-01
4,AK,3321.8,10.460000,20.140000,-33.000000,267.000000,580,570641,94743,665384,60.9,64.9,-8.660254e-01


In [111]:
dataset_loaded.to_csv('datasets/dataset_cleaned.csv', index=False)

In [118]:
from sklearn.preprocessing import TargetEncoder

encoded_state = TargetEncoder(smooth='auto').fit_transform(pd.DataFrame(dataset_loaded['STATE']), dataset_loaded['TOTAL_FIRE_SIZE'])
print(len(encoded_state))
pd.DataFrame(encoded_state).head()


9632


,0
0,114681.668043
1,116110.429778
2,116110.429778
3,135099.898072
4,116110.429778


In [121]:
dataset_loaded['STATE'] = encoded_state
dataset_loaded.head()

,STATE,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE,MONTH_SIN
0,114681.668043,10.1,22.187599,36.501526,-29.515843,282.508235,580,570641,94743,665384,60.9,64.9,8.660254e-01
1,116110.429778,4309.5,5.360000,44.170000,-56.000000,233.000000,580,570641,94743,665384,60.9,64.9,5.000000e-01
2,116110.429778,86460.6,14.030000,39.270000,-17.000000,322.000000,580,570641,94743,665384,60.9,64.9,1.224647e-16
3,135099.898072,48578.5,16.780000,30.910000,28.000000,300.000000,580,570641,94743,665384,60.9,64.9,-5.000000e-01
4,116110.429778,3321.8,10.460000,20.140000,-33.000000,267.000000,580,570641,94743,665384,60.9,64.9,-8.660254e-01


In [122]:
dataset_loaded.to_csv('datasets/dataset_final_1.csv', index=False)